<a href="https://colab.research.google.com/github/neurologic/Neurobiology-Movement/blob/main/executable/NotebookColab_Kinematics_WBDSdataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kinematics and Kinetics of Walking

We can’t understand anything about how the nervous system organizes and controls a movement until we can understand the movement itself and the biomechanics of the body controlled by the nervous system. 

Kinematics is the study and quantification of how objects move. In working through the contents of this notebook, you will learn more about movement and how it is measured and analyzed. You will also practice using data processing and visualization techniques common to neuroscience, physics, biology, and many data science professions. 

Specifically, you will learn about how (some) human bodies walk in the context of current standard practices in the fields of biomechanics. The [dataset you will explore](https://doi.org/10.6084/m9.figshare.5722711) was published by [Fukuchi et al (2018)](https://peerj.com/articles/4640/). They placed markers placed at specific locations along the body were tracked during walking (either overland or on a treadmill). Note that the nomenclature for the marker points in this dataset vary slightly from [the marker placement resource](https://www.sciencedirect.com/science/article/pii/S0966636207000124?via%3Dihub) referenced in Fukuchi et al (2018). See marker key in slides from class. Force plates were used to quantify ground contact forces during walking ("kinetics").

<img src='https://github.com/neurologic/Neurobiology-Movement/blob/main/images/kinematics_markers_Fukuchi.png?raw=true' width="600" >


Your work on this notebook will scaffold discussions throughout the course about kinematics of a variety of movements, the challenges faced by the nervous system in controlling movement, and emerging technologies that are revolutionizing ways of quantifying movement. 



# Setup

Complete the following preparatory steps to set up the Colab notebook environment for your work.


In [ ]:
#@title {display-mode: "form"}

#@markdown RUN this cell to set up the notebook (import packages, define functions, etc)
# In Python, anything with a "#" in front of it is code annotation,
# and is not read by the computer.
# You can run a cell (this box) by pressing ctrl-enter or shift-enter.
# You can also run a cell by clicking the play button in the menu bar 
# at the top of the page (single right arrow, not double).
# Click in this cell and then press shift and enter simultaneously.
# This print function below allows us to generate a message.
print('Nice work!')

# No need to edit anything in this code cell
#################################

import numpy as np
import pandas as pd
from scipy import ndimage
from scipy.signal import find_peaks
from copy import deepcopy
import math
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
import csv
from scipy.signal import hilbert,medfilt,resample
from scipy.io import wavfile
from sklearn.decomposition import PCA
import scipy
import seaborn as sns
from datetime import datetime,timezone,timedelta
pal = sns.color_palette(n_colors=15)
pal = pal.as_hex()


# datafolder = "/content/drive/Shareddrives/BIOL358/Data/WBDSascii"
# datafolder = "/content/drive/MyDrive/Classroom/BIOL358: Motor Systems/Data/WBDSascii"
# datafolder = '/Users/kperks/OneDrive - wesleyan.edu/Teaching/MotorSystems_SP22/Data/WBDSascii/'

fs_grf = 300
fs_mkr = 150

def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)
    
print('Task completed at ' + str(datetime.now(timezone(-timedelta(hours=5)))))

In [ ]:
#@title {display-mode: "form"}

#@markdown RUN this cell to download and unzip the raw data from "figshare".
#@markdown Note that this step may take a few minutes. If the "run cell" button is still spinning, then it is still working.

!wget https://figshare.com/ndownloader/files/10058986

# use --no-check-certificate to ignore SSL certificate errors that might occur on websites with an expired SSL certificate

!unzip -q /content/10058986 -d /content/WBDSascii 

datafolder = "/content/WBDSascii"

print('Task completed at ' + str(datetime.now(timezone(-timedelta(hours=5)))))

The dataset should now be located in your Colab file directory in a folder called "WBDSascii". Check that it is there before proceeding.

# Raw Data

## Activate menus and select data

Run the code cell below to activate the dropdown menus for selection of raw data from a particular person, environment, and speed (note different selection options based on the environment chosen).
  > NOTE: you should only need to run the code cell below once per active colab session to activate the dropdown menus. If you can change your selections, it is activated. You can change your selections at any time and load your selection using the "Load DATASET" code cell.

In [ ]:
#@title {display-mode: "form"}

#@markdown Activate Dropdown Menus if needed.
 
w_subj = widgets.Select(
    options=['select person (01-42)'] + [format(x,'02d') for x in range(1,43,1)],
    value='select person (01-42)',
    # rows=10,
    disabled=False
)
w_env = widgets.Select(
    options=['select environment','Overland','Treadmill'],
    value='select environment',
    rows=3,disabled=False
)
w_Ospeed = widgets.Select(
    options=['select speed for overland','Comfortable','Fast','Slow'],
    value='select speed for overland',
    rows=4,disabled=False
)
w_Tspeed = widgets.Select(
    options=['select speed for treadmill (01-08)'] + [format(x,'02d') for x in range(1,9,1)],
    value='select speed for treadmill (01-08)',
    rows=5,disabled=False
)
display(w_subj,w_env,w_Ospeed,w_Tspeed)
print('Dropdowns generated at ' + str(datetime.now(timezone(-timedelta(hours=5)))))

## Load DATASET

Once you have made your selection, then load the dataset.


In [ ]:
#@title {display-mode: "form"}

#@markdown Run this code cell to load the data you have selected
#@markdown >Note: If you change your selections (different person, environment, or speed)
#@markdown , re-run this cell.


if w_env.value=='Overland':
    speed = w_Ospeed.value[0]
    trial = '01'
if w_env.value=='Treadmill':
    speed = w_Tspeed.value
    trial = ''

filename = 'WBDS' + str(w_subj.value) + 'walk' + w_env.value[0] + trial + speed 

filepath = Path(datafolder) / (filename + 'mkr.txt')
if filepath.exists():
    dataframe = pd.read_csv(filepath, sep='\t')
    print('Kinematic Data loaded at ' + str(datetime.now(timezone(-timedelta(hours=5)))))

if filepath.exists()==False:
    print('Data selection does not exist. Check selections and correct errors.')
    print('Task not yet completed, but cell ran at ' + str(datetime.now(timezone(-timedelta(hours=5)))))    

filepath_grf = Path(datafolder) / (filename + 'grf.txt')
if filepath_grf.exists():
    grf_trial = pd.read_csv(filepath_grf,sep='\t')
    grf_trial['Time']/=fs_grf
    print('Kinetic Data loaded at ' + str(datetime.now(timezone(-timedelta(hours=5)))))

if filepath_grf.exists()==False:
    print('Data selection does not exist. Check selections and correct errors.')
    print('Task not yet completed, but cell ran at ' + str(datetime.now(timezone(-timedelta(hours=5)))))

w_markers = widgets.SelectMultiple(
    options=list(dataframe.columns)[1:],
    rows=15)

w_fp = widgets.SelectMultiple(
    options=list(grf_trial.columns)[1:],
    rows=10)
print('')
print('')
print('Select which kinematics variables you want to plot from the following list:')
display(w_markers)
print('')
print('')
print('Select which kinetic variables you want to plot from the following list:')
display(w_fp)
print('Dropdown menus created at ' + str(datetime.now(timezone(-timedelta(hours=5)))))






## Kinematics 

You can select different kinematics variables and re-make the following plot as many times as you want without re-loading the dataset. 

Follow your curiosities about what kinematics are related to each other and in what ways. There are many to choose from. Explore both ‘ipsilateral’ and ‘contralateral’ comparisons. 

In [ ]:
#@title {display-mode: "form"}

#@markdown Run this code cell to plot the selected kinematics variables

fig = go.Figure()
if len(w_markers.value)>0:
    for marker in list(w_markers.value):
        fig.add_trace(go.Scatter(x = dataframe['Time'], y = dataframe[marker],name=marker))
    print('Task completed at ' + str(datetime.now(timezone(-timedelta(hours=5)))))

if len(w_markers.value)==0:
    print('Need to select markers to plot')
    print('Task not completed yet, but cell run at ' + str(datetime.now(timezone(-timedelta(hours=5)))))
    
fig.update_layout(xaxis_title="time(seconds)", yaxis_title='position (centimeters)',width=800, height=500)

## Kinetics

Select a subset of ground force measurements to examine. Note that the treadmill environment has more ground force measurement data than overland. 

In [ ]:
#@title {display-mode: "form"}

#@markdown Run this code cell to plot the ground reaction force measurements you selected.

fig = go.Figure()
if len(w_fp.value)>0:
    for fp in list(w_fp.value):
        fig.add_trace(go.Scatter(x = grf_trial['Time'], y = grf_trial[fp],name=fp))
    print('Plot created at ' + str(datetime.now(timezone(-timedelta(hours=5)))))

if len(w_fp.value)==0:
    print('Need to select markers to plot')
    print('Task not completed, but cell run at ' + str(datetime.now(timezone(-timedelta(hours=5)))))

fig.update_layout(xaxis_title="time(seconds)", yaxis_title='force',width=800, height=500)



## Identify individual gait cycle trials

**Select data from the treadmill condition.**

In this section, you will transform the continuous measurements into trials and look at trial-to-trial variability. Each trial will be defined by the onset of a gait cycle. The onset of a gait cycle is determined based on kinetic data (ground reaction forces measured by the force plates).

Based on the ground force measurements you just plotted, use your mouse hover to determine the start times of 3 trials. 

In the code cell below, there is a form with spots to create a list of trial start times and assign a variable for the approximate gait cycle period. Single click just above the line (or highlight the text) to edit the form entries. After filling in the necessary information, run the code cell to store the information into temporary memory. 

In [ ]:
#@markdown Type a list of trial times below. 
#@markdown >The formatting of the list must be trial start times (in seconds)
#@markdown separated by commas. The whole list must be surrounded by hard brackets. 
#@markdown For example: ```[1.24,5.67,8.98]```
trials = [None] #@param 

#@markdown Type the gait cycle period below (in seconds). 
period =  None #@param 

#@markdown After you have created a list of trial start times and specified a period,
#@markdown run this cell to store the values in memory for the next task. 

offset = 0.5
persamps = int(period*fs_grf)+int(offset*fs_grf)
trial_inds = [int((t-offset)*fs_grf) for t in trials]
xtime = np.linspace(-offset,period,persamps)

print('Tasks completed at ' + str(datetime.now(timezone(-timedelta(hours=5)))))



### Plot trial data

Run the cells below to plot data on each trial that you specified. 

When plotting kinematics data, make sure you have selected specific marker locations from the dropdown menu in the "Load DATASET" section.


In [ ]:
#@title {display-mode: "form"}

#@markdown Run this code cell to plot the kinetic measurements on each trial overlaid.

offset = 0.5
persamps = int(period*fs_grf)+int(offset*fs_grf)
trial_inds = [int((t-offset)*fs_grf) for t in trials]
xtime = np.linspace(-offset,period,persamps)

fig = go.Figure()

if len(w_fp.value)>0:
    for fp in list(w_fp.value):
      for t in trial_inds:
        fig.add_trace(go.Scatter(x = xtime, y = grf_trial[fp][t:t+persamps],name=fp+' at trial ' +str(round(t/fs_grf,2)+offset)))
    print('Task completed at ' + str(datetime.now(timezone(-timedelta(hours=5)))))

if len(w_fp.value)==0:
    print('Need to select markers to plot')
    print('Task not completed, but cell run at ' + str(datetime.now(timezone(-timedelta(hours=5)))))

fig.update_layout(xaxis_title="time(seconds)", yaxis_title='force',width=800, height=500)



In [ ]:
#@title {display-mode: "form"}

#@markdown Run this code cell to plot the kinematics measurements on each trial overlaid.

offset = 0.5
persamps = int(period*fs_mkr)+int(offset*fs_mkr)
trial_inds = [int((t-offset)*fs_mkr) for t in trials]
xtime = np.linspace(-offset,period,persamps)

fig = go.Figure()

if len(w_markers.value)>0:
    for marker in list(w_markers.value):
      for t in trial_inds:
        fig.add_trace(go.Scatter(x = xtime, y = dataframe[marker][t:t+persamps],name=str(marker)+' at trial ' +str(round(t/fs_mkr,2))))
    print('Task completed at ' + str(datetime.now(timezone(-timedelta(hours=5)))))

if len(w_markers.value)==0:
    print('Need to select markers to plot')
    print('Task not completed, but cell run at ' + str(datetime.now(timezone(-timedelta(hours=5)))))

fig.update_layout(xaxis_title="time(seconds)", yaxis_title='position',width=800, height=500)


You can go back and select different kinematics variables to plot on each of these trials (remember, no need to reload the dataset unless you choose a different person, environment, or speed).

# Time-normalized (processed) data

Ok. Now that you have a sense of the data, let's look at average and time-normalized data. 
In this step, the data that you will explore is trial-averaged measurements (the average across trials for each. The data is "time-normalized" so that the kinematics and kinetics are a function of the gait cycle (from start to stop - 0 to 100% of the gait cycle) instead of time. 

> Based on the overlaid trial data you just examined, why is time normalization important for an analysis of gait cycle kinematics and kinetics?



## Activate menus and select data

Run the code cell below to activate the dropdown menus for selection of time-normalized data from a particular person, environment, and speed (note different selection options based on the environment chosen).
  > NOTE: you should only need to run the code cell below once per active colab session to activate the dropdown menus. If you can change your selections, it is activated. You can change your selections at any time and load your selection using the "Load DATASET" code cell.

In [ ]:
#@title {display-mode: "form"}

#@markdown Activate Dropdown Menus if needed.
 
 
w_subj_norm = widgets.Select(
    options=['select person (01-42)'] + [format(x,'02d') for x in range(1,43,1)],
    value='select person (01-42)',
    # rows=10,
    disabled=False
)
w_env_norm = widgets.Select(
    options=['select environment','Overland','Treadmill'],
    value='select environment',
    rows=3,disabled=False
)
w_Ospeed_norm = widgets.Select(
    options=['select speed for overland','Comfortable','Fast','Slow'],
    value='select speed for overland',
    rows=4,disabled=False
)
w_Tspeed_norm = widgets.Select(
    options=['select speed for treadmill (01-08)'] + [format(x,'02d') for x in range(1,9,1)],
    value='select speed for treadmill (01-08)',
    rows=5,disabled=False
)
display(w_subj_norm,w_env_norm,w_Ospeed_norm,w_Tspeed_norm)

print('Dropdowns generated at ' + str(datetime.now(timezone(-timedelta(hours=5)))))



## Load DATASET

Once you have made your selection, then load the dataset.


In [ ]:
#@title {display-mode: "form"}

#@markdown Run this code cell to load the dataset you have selected
#@markdown >Note: If you change your dataset selections (different person, environment, or speed)
#@markdown , re-run this cell.


if w_env_norm.value=='Overland':
    speed = w_Ospeed_norm.value[0]

if w_env_norm.value=='Treadmill':
    speed = w_Tspeed_norm.value

filename_norm = 'WBDS' + str(w_subj_norm.value) + 'walk' + w_env_norm.value[0] + speed 

filepath_norm = Path(datafolder) / (filename_norm + 'ang.txt')
ang_norm = pd.read_csv(filepath_norm,sep = '\t')
print('Processed kinematics data loaded.')
w_norm_ang = widgets.SelectMultiple(
    options=list(ang_norm.columns)[1:],
    rows=10)

filepath_norm = Path(datafolder) / (filename_norm + 'knt.txt')
grf_norm = pd.read_csv(filepath_norm,sep = '\t')
print('Processed ground fource data loaded.')
w_norm_grf = widgets.SelectMultiple(
    options=list(grf_norm.columns)[1:],
    rows=10)
print('')
print('')
print('Select which processed kinematics variables to plot from the following list (ctrl + click to select multiple).')
display(w_norm_ang)
print('')
print('Select which processed ground force variables to plot from the following list (ctrl + click to select multiple).')
display(w_norm_grf)



## Kinematics 

You can select different kinematics variables and re-make the following plot as many times as you want without re-loading the dataset. 

In [ ]:
#@title {display-mode: "form"}

#@markdown Run this code cell to plot the selected kinematics variables

fig = go.Figure()

if len(w_norm_ang.value)>0:
    for v in list(w_norm_ang.value):
        fig.add_trace(go.Scatter(x = ang_norm['Time'], y = ang_norm[v],name=v))
    print('Plot created at ' + str(datetime.now(timezone(-timedelta(hours=5)))))
if len(w_norm_ang.value)==0:
    print('Need to select measurements to plot')
    print('Task not completed, but cell run at ' + str(datetime.now(timezone(-timedelta(hours=5)))))
    
fig.update_layout(xaxis_title="percent of gait cycle", yaxis_title='angle (degrees)',width=800, height=500)

## Kinetics

Select a subset of ground force measurements to examine. Note that the treadmill environment has more ground force measurement data than overland. 

In [ ]:
#@title {display-mode: "form"}

#@markdown Run this code cell to plot the ground reaction force measurements you selected.

fig = go.Figure()

if len(w_norm_grf.value)>0:
    for v in list(w_norm_grf.value):
        fig.add_trace(go.Scatter(x = grf_norm['Time'], y = grf_norm[v],name=v))
    print('Plot created at ' + str(datetime.now(timezone(-timedelta(hours=5)))))
if len(w_norm_grf.value)==0:
    print('Need to select calculations to plot')
    print('Task not completed, but cell run at ' + str(datetime.now(timezone(-timedelta(hours=5)))))
    
fig.update_layout(xaxis_title="percent of gait cycle", yaxis_title='force',width=800, height=500)



### TO PLOT BOTH KINEMATICS AND KINETICS ON SAME PLOT...
# #@markdown Run this cell to plot your kinematic and kinetic selections.
# fig = make_subplots(rows=2, cols=1,
#                     shared_xaxes=True,
#                     vertical_spacing=0.02)

# if len(w_norm_ang.value)>0:
#     for v in list(w_norm_ang.value):
#         fig.add_trace(go.Scatter(x = ang_norm['Time'], y = ang_norm[v],name=v),row=1,col=1)
# if len(w_norm_ang.value)==0:
#     print('Need to select markers to plot')

# if len(w_norm_grf.value)>0:
#     for v in list(w_norm_grf.value):
#         fig.add_trace(go.Scatter(x = grf_norm['Time'], y = grf_norm[v],name=v),row=2,col=1)
# if len(w_norm_grf.value)==0:
#     print('Need to select markers to plot')

# print('Data upload completed at ' + str(datetime.now(timezone(-timedelta(hours=5)))))

# fig.update_layout(xaxis2_title="percent gait cycle", yaxis_title='degrees',yaxis2_title='N/kg',
#                   width=800, height=500)

### Ground reaction force vector direction (2D)

In [ ]:
#@markdown Run this cell to plot the ground reaction force vector direction 
#@markdown >Note: vector direction calculated in 2D (x,y) plane only.

fig = go.Figure()
rho,phi = cart2pol(grf_norm['RGRFX'],grf_norm['RGRFY'])
fig.add_trace(go.Scatter(x = grf_norm['Time'], y = np.degrees(phi),
                         name='right',mode='markers'))
rho,phi = cart2pol(grf_norm['LGRFX'],grf_norm['LGRFY'])
fig.add_trace(go.Scatter(x = grf_norm['Time'], y = np.degrees(phi),
                         name='left',mode='markers'))
fig.update_layout(xaxis_title="percent gait cycle", yaxis_title='degrees',
                  width=800, height=400)


### Ground reaction force vector direction and magnitude

In [ ]:
#@markdown <b>Task:</b> Run this cell to make a polar plot 
#@markdown of the ground reaction force vector direction and magnitude (in 2D x,y plane).
fig = go.Figure()
fig.add_trace(go.Scatterpolar(
        r = rho,
        theta = np.degrees(phi),
        mode = 'markers',
        marker_color='black'))
fig.update_layout(width=500, height=500)

<hr>
<br>

## NICE WORK!

<br>


<hr>

For users not in this course:
This notebook assumes that you have access to the BIOL358 shared class drive in which the dataset from [Fukuchi et al (2018)](https://peerj.com/articles/4640/) has been stored. To change the filepath, or aquire the dataset via direct download (if that is possible), edit the first code cell in Step #2 ("TASK: RUN this cell to set up the notebook (import packages, etc)" and change the datafolder path variable. And/or edit the code cells in which the data is imported. 